Выгрузка данных о компаниях
---
Из базы данных выгружается список id работодателей и по ним строится запрос к API для получения полной информации

Для этого:
- Из бд выгружаются уникальные значения employer_id
- В несколько потоков осуществляется доступ к API

In [1]:
%%time
from services.db_service import DbService
from data_parsers.hhParser import HhParser
import threading

db_service = DbService()
employers = db_service.execute_script('SELECT DISTINCT employer_id FROM vacancies WHERE employer_id is not NULL')

parser = HhParser()
def load_employers(empl):
    data = parser.get_employers_info(empl)
    db_service.save_employers(data)

threads = []
size = int(len(employers)/4)
threads.append(threading.Thread(target=load_employers, args=(employers[:size*1],)))
threads.append(threading.Thread(target=load_employers, args=(employers[size*1:size*2],)))
threads.append(threading.Thread(target=load_employers, args=(employers[size*2:size*3],)))
threads.append(threading.Thread(target=load_employers, args=(employers[size*3:],)))

for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

Выгрузка нанимателей:   0%|          | 0/5874 [00:00<?, ?it/s]

Выгрузка нанимателей:   0%|          | 0/5874 [00:00<?, ?it/s]

Выгрузка нанимателей:   0%|          | 0/5874 [00:00<?, ?it/s]

Выгрузка нанимателей:   0%|          | 0/5874 [00:00<?, ?it/s]

Запись в БД:   0%|          | 0/5865 [00:00<?, ?it/s]

Запись в БД:   0%|          | 0/5864 [00:00<?, ?it/s]

Запись в БД:   0%|          | 0/5866 [00:00<?, ?it/s]

Запись в БД:   0%|          | 0/5867 [00:00<?, ?it/s]

Wall time: 31min 54s
